# Projektarbeit Machine Learning und Deep Learning (Gruppe 7)
---

Chris-Josip Puclin, Natalie Rudolf, Tina Schiele

**Aufgabenstellung**

* Datensatz: MVTec AD (Quelle: https://www.mvtec.com/company/research/datasets/mvtec-ad)
* Datensatzbeschreibung: „comprises 15 categories with 3629 images for training and validation and 1725 images for testing.The training set contains only images without defects. The test set contains both: images containing various types of defects and defect-free images.“
* Klassifikationsproblem für das gesamte Bild (image-level classification); z.B.: Liegt ein Fehler im aktuellen Bild vor? Um welches Objekt handelt es sich? etc.
* Hierzu müssen zunächst die Metainformationen (Labels) angepasst werden, sowie Training- und Testdatensatz neu gemischt werden.

**Ideen:**
* Stand der Forschung sichten (es gibt etliche Papers, wir möchten uns auf image-level classification fokussieren)
* Man könnte z.B. hierarchische Klassifikation einsetzen: 1) welches Objekt, 2) IO oder NIO
* Man könnte das Problem als two-class classification problem ansehen
* Oder als multi-class classification Problem (Objekt + IO/NIO => 30 Klassen)


Das nachfolgende Projekt orientiert sich in seinem Aufbau und seiner Struktur am CRISP-DM Modell.

## 1. Project Understanding
---
*Papers, die sich mit image-level classification beschäftigen*

*in diesem Abschnitt kein Code, sondern qualitative Beschreibung, was wir tun wollen*

## 2. Data Understanding
---

*Datensatzbeschreibung: „comprises 15 categories with 3629 images for training and validation and 1725 images for testing.The training set contains only images without defects. The test set contains both: images containing various types of defects and defect-free images.“*

## 3. Data Preparation 
---

## 4. Modeling
---

## 5. Evaluation
---

## 6. Deployment
---

Grundsätzlich entfällt dieser Schritt, da der entwickelte Algorithmus nicht produktiv genutzt werden soll. Die Ergebnisse werden in der Abschlusspräsentation vorgestellt.